In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


True

### Karger Books Para

In [4]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [5]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para/bart"'

output_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_karger_books_para_train"'

log_dir = bucket_dir + '/logs'

In [6]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path, cache_dir=cache_dir)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.length_penalty = config.LENGTH_PENALTY
model_config.no_repeat_ngram_size = config.NO_REPEAT_NGRAM_SIZE

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['length_penalty'] = config.LENGTH_PENALTY
model_config.task_specific_params['summarization']['no_repeat_ngram_size'] = config.NO_REPEAT_NGRAM_SIZE
model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

##### Fine tune

In [9]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--fp16 \
--learning_rate 3e-5 --label_smoothing 0.1 \
--sortish_sampler --freeze_embeds --adafactor \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--test_max_target_length $config.ONE_BULLET_MAX_LEN \
--do_train \
--num_train_epochs 10 \
--logging_steps 200 --save_steps 1500 --save_total_limit 3 \
--per_device_train_batch_size 4 --per_device_eval_batch_size 4 \
--do_eval --evaluation_strategy steps --eval_steps 1500 --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

01/14/2021 15:57:36 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
01/14/2021 15:57:36 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_train', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Jan14_15-57-36_ip-172-31-39-35', logging_first_step=False, logging_steps=200, save_steps=1500, save_total_limit=3, no_

01/14/2021 15:58:08 - INFO - __main__ -   *** Train ***
[INFO|trainer.py:703] 2021-01-14 15:58:08,126 >> ***** Running training *****
[INFO|trainer.py:704] 2021-01-14 15:58:08,126 >>   Num examples = 2046
[INFO|trainer.py:705] 2021-01-14 15:58:08,126 >>   Num Epochs = 10
[INFO|trainer.py:706] 2021-01-14 15:58:08,126 >>   Instantaneous batch size per device = 4
[INFO|trainer.py:707] 2021-01-14 15:58:08,126 >>   Total train batch size (w. parallel, distributed & accumulation) = 4
[INFO|trainer.py:708] 2021-01-14 15:58:08,126 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:709] 2021-01-14 15:58:08,126 >>   Total optimization steps = 5120
/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_train
[INFO|integrations.py:371] 2021-01-14 15:58:08,129 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wan

 59%|█████▊    | 3000/5120 [17:31<14:58,  2.36it/s][INFO|trainer.py:1412] 2021-01-14 16:15:40,502 >> ***** Running Evaluation *****
[INFO|trainer.py:1413] 2021-01-14 16:15:40,502 >>   Num examples = 266
[INFO|trainer.py:1414] 2021-01-14 16:15:40,502 >>   Batch size = 4


 99%|█████████▊| 66/67 [01:03<00:00,  1.03it/s]
                                                   {'eval_loss': 587.0930786132812, 'eval_rouge1': 32.7734, 'eval_rouge2': 14.9052, 'eval_rougeL': 26.4432, 'eval_rougeLsum': 27.137, 'eval_gen_len': 39.4, 'epoch': 5.859375}
100%|██████████| 67/67 [01:04<00:00,  1.25it/s]
                                               
[INFO|trainer.py:1226] 2021-01-14 16:16:47,361 >> Saving model checkpoint to /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_train/checkpoint-3000
[INFO|configuration_utils.py:289] 2021-01-14 16:16:47,510 >> Configuration saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_train/checkpoint-3000/conf

 99%|█████████▊| 66/67 [01:01<00:00,  1.05it/s]
                                                   {'eval_loss': 607.7008666992188, 'eval_rouge1': 33.2772, 'eval_rouge2': 14.8383, 'eval_rougeL': 26.6318, 'eval_rougeLsum': 27.2894, 'eval_gen_len': 39.7, 'epoch': 8.7890625}
100%|██████████| 67/67 [01:03<00:00,  1.24it/s]
                                               
[INFO|trainer.py:1226] 2021-01-14 16:26:24,995 >> Saving model checkpoint to /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_train/checkpoint-4500
[INFO|configuration_utils.py:289] 2021-01-14 16:26:25,147 >> Configuration saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_train/checkpoint-4500/config.json
[INFO|modeling_utils.py:814] 2021-01-14 16:27:08,420 >> Model weights saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_train/checkpoint-4500/pytorch_model.bin
 94%|█████████▍| 4800/5120 [30:33<02:16,  2.34it/s]'epoch': 9.375}


##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--bs 2 \
--length_penalty $config.LENAGTH_PENALTY \
--no_repeat_ngram_size $config.NO_REPEAT_NGRAM_SIZE \
--num_beams $config.NUM_BEAMS \
--dump-args